## Lesson 2: Pandas Basics
author = [mguan]

This is the second lesson in the Python 101 series. By the end of this series, you should have a solid foundational grasp of `pandas` - a Python library used for data manipulation, and how to use it to read and manipulate data

### What is Pandas Dataframe
The building block of all work with `pandas` is the dataframe. On the surface, a dataframe is identical to a csv file, the output of a sql query, etc.

<img src=https://cdn-images-1.medium.com/max/1800/1*6p6nF4_5XpHgcrYRrLYVAw.png width="700">
<br>

Thinking about it in terms of python object though, `pandas` takes many of the features of the basic data types that we discussed earlier and aggregates them into a 2+ dimensional dataframe.

The building block of pandas dataframes are `numpy` arrays. `numpy` arrays are essentially indexed lists of one datatype that are vectorized and stored more efficiently than base python lists. These are the columns of the dataframe. A dataframe contains many of these arrays sharing the same index values (0, 1, 2, 3 ... on the left).

Like a dictionary, pandas dataframes are indexed - however it is indexed in 2 dimensions - the x (row) and y (column) axis of the dataframe. With this structure, we can return a row of data (values across multiple numpy arrays that share the same index value) or a column of data (one individual `numpy` array)


First lets import pandas. Note that we are aliasing the package as `pd` for ease of use

In [1]:
import pandas as pd

Next we can create our first dataframe. We will go over the two most common ways to create a dataframe. The first is from a basic dictionary of lists.

Take a look at the dictionary below. The __keys__ ("id" and "value") get converted to columns, and the __values__ in each list get converted to the first, second and third rows of data for the respective key.

In [2]:
df_dict = {
    "id": [1, 2, 3],
    "group": ["a", "b", "b"]
}

Our dataframe is below. Note that we must pass in a list to the `index` argument to specify the index values. These index values are what will allow us to reference a specific row of data, and extract or subset the dataframe.

In [3]:
pd.DataFrame(df_dict, index=[0, 1, 2])

,id,group
0,1,a
1,2,b
2,3,b


The second way is to build the dataframe using a nested dictionary. The outermost key of the dictionaries correspond to the index values, and each inner dictionary contains the column name mapped to the value for the respective index.

The below code block is used to generate the exact same dictionary as before

In [4]:
df_dict = {0: {"id": 1,
               "group": "a"},
           1: {"id": 2,
               "group": "b"},
           2: {"id": 3,
               "group": "b"}}

In [5]:
df = pd.DataFrame.from_dict(df_dict, orient="index")
df

,id,group
0,1,a
1,2,b
2,3,b


### Pulling Data From SQL

to pull data from sql, you can initialize a database engine using `sqlalchemy` and pass it directly into the method `pd.read_sql`. Below is some mock code **(this won't actually work)** that you can use. We are parameterizing the query with a lsit in this example

In [ ]:
# obviously don't do this in plaintext in production

from sqlalchemy import create_engine

engine = create_engine(f"redshift+psycopg2://{USER}:{PASS}@{DATABASE_URL}:{HOST}/{DATABASE}")

In [ ]:
query = """
SELECT * 
FROM 
schema.table 
WHERE
ids in %(ids)s
LIMIT 3
"""

In [ ]:
ids = ["1", "2", "3"]

In [ ]:
df = pd.read_sql(query, engine, params={"ids": tuple(ids)})

    lower(e.name) like '%%{0}%%')

### Take a Quick Look at the Data 

Now lets go over a few functions to do quick spot checks on the data

The `head` method returns the first 5 rows of the dataframe. You can also pass in an integer of your choice to return that many rows

In [6]:
df.head(3)

,id,group
0,1,a
1,2,b
2,3,b


`tail` does the opposite

In [7]:
df.tail(3)

,id,group
0,1,a
1,2,b
2,3,b


`transpose` swaps columns and rows, which can make it easier to view very wide dataframes. __NOTE__ that in the below code snipped I am using head and then transpose - pandas allows chaining of methods like this.

In [8]:
df.head(1).transpose()

,0
id,1
group,a


`describe` gives some basic summary statistics for integer and float columns

In [9]:
df.describe()

,id
count,3.0
mean,2.0
std,1.0
min,1.0
25%,1.5
50%,2.0
75%,2.5
max,3.0


`dtypes` gives information on the datatype of each column in the dataframe. Note that object means string

In [10]:
df.dtypes

id        int64
group    object
dtype: object

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 2
Data columns (total 2 columns):
id       3 non-null int64
group    3 non-null object
dtypes: int64(1), object(1)
memory usage: 72.0+ bytes


`columns` returns a list of columns

In [12]:
list(df.columns)

['id', 'group']

`shape` returns the dimensions of the dataframe as a tuple (rows, columns)

In [13]:
df.shape

(3, 2)

### Basic Data Manipulation 

Beyond the basic data investigation methods above, there are a ton of built in methods that allow us to perform all kinds of data manipulation. Basically anything that you were doing in sql before, you can do in pandas in a much cleaner and more intuitive way

__Select Specific Columns__

In [14]:
test = df[["id"]]
print(test.shape)
test.head(3)

(3, 1)


,id
0,1
1,2
2,3


__Extract a Values by Index__

In [15]:
my_value = df.loc[0, "id"]

print(my_value, type(my_value))

1 <class 'numpy.int64'>


__Extract Unique Values By Column__

In [16]:
unique_array = test["id"].unique()
unique_list = unique_array.tolist()

In [17]:
print(unique_list)

[1, 2, 3]


In [18]:
type(unique_array)

numpy.ndarray

In [19]:
type(unique_list)

list

__Filter by Values in a Column__

Simple Filter


    filtered_df = df[filter_condition]



In [20]:
filtered = df[df["id"] == 1]
filtered

,id,group
0,1,a


In [21]:
filtered = df[df["id"].isin([1,2])]
filtered

,id,group
0,1,a
1,2,b


Combining filters with **AND**

    filtered_df = df[(filter_condition1) & (filter_condition2)]

In [22]:
filtered = df[(df["id"] == 1) & (df["group"] == "a")]
filtered.shape

(1, 2)

In [23]:
filtered

,id,group
0,1,a


**OR**

    filtered_df = df[(filter_condition1) | (filter_condition2)]

In [24]:
filtered = df[(df["id"] == 1) | (df["id"] == 3)]
filtered

,id,group
0,1,a
2,3,b


__Drop Duplicates__

In [25]:
test = test.drop_duplicates()
print(test.shape)

(3, 1)


__Fill Null Values__

In [26]:
df = df.fillna("None")

__Rename Columns__

Name map syntax:

    {'old':'new'}


In [27]:
df = df.rename(columns={"value": "metric"})

In [28]:
df.columns

Index(['id', 'group'], dtype='object')

__Convert Datatypes__

In [29]:
df["id"] = df["id"].astype(str)

In [30]:
# "object" dtype

df.dtypes["id"]

dtype('O')

__Join Data__

In [31]:
df2 = pd.DataFrame({"id": ["1","2","3"], "value": [5, 6, 7]})

In [32]:
df2

,id,value
0,1,5
1,2,6
2,3,7


In [33]:
joined = pd.merge(df, df2, on="id", how="inner")
joined

,id,group,value
0,1,a,5
1,2,b,6
2,3,b,7


In [34]:
len(df.columns)

2

In [35]:
len(joined.columns)

3

__Union Data__

In [36]:
j1 = joined.head(1)
j2 = joined.tail(1)

In [37]:
unioned = pd.concat([j1, j2])

In [38]:
unioned

,id,group,value
0,1,a,5
2,3,b,7


In [39]:
unioned.reset_index(drop=True)

,id,group,value
0,1,a,5
1,3,b,7


__Aggregate Data (groupby)__

In [40]:
joined.groupby(["group"]).agg({"value":["mean", "sum"]})

value    
       mean sum
group          
a       5.0   5
b       6.5  13

In [41]:
joined.groupby(["group"]).count()[["value"]]

,value
group,
a,1
b,2


__Sort Data__

In [42]:
sorted_df = df.sort_values("id")

In [43]:
sorted_df.head(3)

,id,group
0,1,a
1,2,b
2,3,b


__Pivot Data__

Useful for transforming data whilst also applying an aggregation function (think Pivot Table in Excel).

In [44]:
joined.head(3)

,id,group,value
0,1,a,5
1,2,b,6
2,3,b,7


In [45]:
pivoted = joined.pivot_table(values=["value"], 
               index=["group"], 
               columns=["id"], 
               aggfunc="sum")

In [46]:
pivoted

value          
id        1    2    3
group                
a       5.0  NaN  NaN
b       NaN  6.0  7.0

Renaming columns here to flatten the MultiIndex above.

In [47]:
pivoted.columns = ["-".join(str(i) for i in tup) for tup in pivoted.columns.to_series()]

In [48]:
pivoted.reset_index()

,group,value-1,value-2,value-3
0,a,5.0,NaN,NaN
1,b,NaN,6.0,7.0


__Melt Data__
* Turn a column into an indexed row
* useful for changing the data format from wide to long for visualization
    * ie, turning multiple columns into one label column and one value column
* no aggregation

In [49]:
pd.melt(joined, id_vars=["id"], value_vars=["group", "value"]).sort_values(
    "id").drop_duplicates().reset_index(drop=True)

,id,variable,value
0,1,group,a
1,1,value,5
2,2,group,b
3,2,value,6
4,3,group,b
5,3,value,7


__Apply A Custom Function to a Column__

In [50]:
joined.head(3)

,id,group,value
0,1,a,5
1,2,b,6
2,3,b,7


In [51]:
joined["clean_group"] = joined["group"].apply(lambda x: x.replace("a", "A") if 'a' in x else x)

In [52]:
joined.head(3)

,id,group,value,clean_group
0,1,a,5,A
1,2,b,6,b
2,3,b,7,b
